In [18]:
# Practice problem: Scraping amazon.in to get shoe price data 
import pprint,requests,bs4
import pandas as pd

In [30]:
#constants
url = "https://www.amazon.in/s/ref=nb_sb_noss?url=search-alias%3Daps&field-keywords=sport+shoes"
brand_select_class = ".a-size-base-plus.a-color-base"
description_select_class= ".a-size-base-plus.a-color-base.a-text-normal"
price_select_class=".a-price-whole"

In [69]:
#column_to_class_mapping= {'Brand':brand_select_class,'Description':description_select_class,'Price':price_select_class}
column_to_class_mapping= {'Brand':brand_select_class}

In [32]:
def get_data_by_class(classText:any,soup:any):
    return soup.select(classText)

In [67]:
def split_data_by_span_class(classText:any,data:any):
    try: return [str(r).split("</span>")[0].split('<span class="'+classText+'">')[1] for r in data]
    except:print ([str(r).split("</span>")[0].split('<span class="'+classText+'">') for r in data])


In [95]:
def scrape_website(url):
    req = requests.get(url)
    # create a bs4 object
    # To avoid warnings, provide "html5lib" explicitly
    #html.parser
    return bs4.BeautifulSoup(req.content, "lxml")


In [73]:
def get_data_frame_from_columns(column_to_class_map,scrape_response):
    df = pd.DataFrame()
    for column,class_name in column_to_class_map.items():
        data_by_class = get_data_by_class(class_name,scrape_response)
        actual_data = split_data_by_span_class(class_name.replace("."," ").lstrip(),data_by_class)
        df[column] = pd.Series(actual_data)
       
    return df

In [155]:
#Actual Run
response = scrape_website(url)
df = pd.DataFrame(columns=['Id','Brand','Description','Price','Discount','Rating','Total_Reviews','Free_Delivery'])
df.set_index('Id')

search = response.find_all("div",  attrs={"id": 'search'})[0]
product_spans = search.find_all("span",attrs={"class": 'celwidget slot=MAIN template=SEARCH_RESULTS widgetId=search-results'})
product_id=1
for product in product_spans:
    try:
        brand = product.find("span",attrs={"class":'a-size-base-plus a-color-base'}).string
        description = product.find("span",attrs={"class":'a-size-base-plus a-color-base a-text-normal'}).string
        price = product.find("span",attrs={"class":'a-price a-text-price'}).find("span",attrs={"aria-hidden":'true'}).string.replace(',','').replace('₹','')
        discount  = product.find("span",attrs={"class":'a-price'}).find("span",attrs={"aria-hidden":'true'}).find("span",attrs={"class":'a-price-whole'}).string.replace(',','')
        rating  = float(product.find("span",attrs={"class":'a-icon-alt'}).string.split("out")[0].strip())
        num_of_reviews  = product.find("span",attrs={"class":'a-size-base'}).string.replace(',','')
        delivery_content  = product.find("div",attrs={"class":'a-row a-size-base a-color-secondary s-align-children-center'}).string
        is_free_deliver = delivery_content.lower().count("free delivery") >0 
    except AttributeError:

           is_free_deliver = False
    #row = pd.DataFrame.from_dict({'Brand':brand,'Description':description,'Price':price,'Discount':discount,'Rating':rating,'Number_Of_Reviews':num_of_reviews,'Delivery':delivery_content})
    a_row = [product_id,brand,description,price,discount,rating,num_of_reviews,is_free_deliver]
    #df= pd.concat([df,a_row],ignore_index=True)
    df.loc[len(df)] = a_row
    product_id +=1


In [156]:
print(df)


    Id                Brand  \
0    1                CLYMB   
1    2                Sparx   
2    3                Sparx   
3    4                Sparx   
4    5               Bourge   
5    6               ALICON   
6    7          A ANTIC MOP   
7    8  WORLD WEAR FOOTWEAR   
8    9               Ethics   
9   10  WORLD WEAR FOOTWEAR   
10  11                ASIAN   
11  12                Sparx   
12  13                Sparx   
13  14                Sparx   
14  15                Sparx   
15  16                ASIAN   
16  17  WORLD WEAR FOOTWEAR   
17  18                 SEGA   
18  19        White Walkers   
19  20               Campus   
20  21               Campus   
21  22                Sparx   
22  23              Sklodge   
23  24               T-Rock   
24  25                CLYMB   
25  26                Sparx   
26  27  WORLD WEAR FOOTWEAR   
27  28                 Cogs   
28  29                Sparx   
29  30  WORLD WEAR FOOTWEAR   
30  31               Campus   
31  32  

In [165]:
df_rating_avg = df.groupby(['Brand']).agg('mean')

In [170]:
print(df_rating_avg.sort_values(['Rating'],ascending=False))

                       Rating
Brand                        
Reebok               4.150000
Sparx                4.106667
Kraasa               4.100000
Cogs                 4.100000
Campus               4.066667
Bourge               4.000000
SEGA                 4.000000
ASIAN                3.933333
GoldStar             3.900000
Shoefly              3.700000
WORLD WEAR FOOTWEAR  3.687500
White Walkers        3.500000
Ethics               3.400000
CLYMB                3.350000
A ANTIC MOP          3.300000
Sklodge              3.200000
MOUNTREXO            2.900000
T-Rock               2.900000
ALICON               2.800000


In [171]:
df_products = df.groupby(['Brand']).agg('sum')

In [176]:
df_brand_prod = df.groupby(['Brand','Description'])

In [179]:
print(df_brand_prod['Rating'].mean())

Brand                Description                                                                                                                               
A ANTIC MOP          Black Fashion Sport Men's Running Shoes(901)                                                                                                  3.30
ALICON               Sports Shoes for Men | Lace up Lightweight Shoes for Running, Walking, Gym, Trekking, Hiking & Party Running Shoes for Men's                  2.80
ASIAN                Men's Bullet-02 Walking,Sports,Gym,Running Shoes                                                                                              3.90
                     Men's Wonder-13 Sports Running Shoes                                                                                                          4.00
                     Men's Worldcup-03 Running,Sports,Gym,Phylon Sports                                                                                            3.90
